## Enviroment

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import json
import psycopg2

Conexión a la base de datos mediante las credenciales definidas

In [ ]:

with open("../credentials.json") as f:
    creds = json.load(f)
    
DB_USER = creds["user"]
DB_PASSWORD = creds["password"]
DB_HOST = creds["host"]
DB_PORT = creds["port"]
DB_NAME = creds["database"]


df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False, encoding='ISO-8859-1')
print("Datos del CSV cargados correctamente.")
print(df.head()) 

# Contar el número total de filas en el CSV
total_filas_csv = len(df)
print(f"El archivo CSV tiene un total de {total_filas_csv} filas.")

# Crear conexión a la base de datos ya creada
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_string)

# Crear la tabla en la base de datos y cargar los datos
table_name = "airbnb_data"  # Nombre de la nueva tabla
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"Tabla '{table_name}' creada y datos cargados en la base de datos '{DB_NAME}'.")

# Verificar los datos en la nueva tabla
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT * FROM {table_name} LIMIT 5;"))
    print(f"Primeros 5 registros de la tabla '{table_name}':")
    for row in result:
        print(row)

In [5]:
# Contar el número total de filas en el DataFrame
total_csv_rows = len(df)
print(f"El archivo CSV tiene un total de {total_csv_rows} filas.")

# Verificar el número total de filas en la tabla
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT COUNT(*) FROM {table_name};"))
    total_rows = result.scalar()  # Obtener el valor del conteo
    print(f"La tabla '{table_name}' tiene un total de {total_rows} filas.")

El archivo CSV tiene un total de 102599 filas.
La tabla 'airbnb_data' tiene un total de 102599 filas.


In [ ]:
with engine.connect() as connection:
    # Consulta para obtener los primeros 20 registros de la tabla
    query = f"SELECT * FROM {table_name} LIMIT 20;"
    
    # Cargar los datos en un DataFrame de pandas
    df = pd.read_sql(query, connection)
    
    # Mostrar el DataFrame en formato tabular
    print(f"Primeros 20 registros de la tabla '{table_name}':")
    print(df)

In [ ]:
# Cargar el dataset
df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False, encoding='ISO-8859-1')

# Revisión inicial
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
print(df.info())  # Tipos de datos y valores no nulos
print(df.isnull().sum())  # Conteo de valores nulos por columna